# 参数管理

我们首先看一下具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8), # 第1层：Linear(4, 8)-全连接层，将4维输入映射到8维
                    nn.ReLU(), # 第2层：ReLU()-激活函数，引入非线性
                    nn.Linear(8,1)) # 第3层：Linear(8, 1)-全连接层，将8维映射到1维输出
# 生成形状为 (2, 4) 的随机张量(2个样本，每个样本 4个特征)
X = torch.rand(size=(2, 4)) 
net(X)

tensor([[-0.4223],
        [-0.3488]], grad_fn=<AddmmBackward0>)

参数访问

In [2]:
# 访问Sequential容器中索引为2的模块（从0开始计数）
# .state_dict()：获取该模块的参数字典，包含所有可学习参数（权重和偏置）
# weight  ：形状为 (1, 8) 的权重矩阵；bias  ：形状为 (1,) 的偏置向量
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0937, -0.0804, -0.2165,  0.2342, -0.0191,  0.3365,  0.0420, -0.3358]])), ('bias', tensor([-0.1681]))])


目标参数

In [3]:
# 查看参数类型，type
print(type(net[2].bias))
# 查看参数对象本身，Parameter
print(net[2].bias)
# 查看底层数据张量，Tensor
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1681], requires_grad=True)
tensor([-0.1681])


In [4]:
# 检查网络中第二个全连接层的权重梯度是否为None
'''
net[2]：访问Sequential中索引为2的模块，即nn.Linear(8,1)输出层
.weight：获取该层的权重参数（形状为(1,8)）
.grad：获取该权重的梯度（损失函数对权重的导数，用于梯度下降）
'''
net[2].weight.grad == None

True

一次性访问所有参数

In [5]:
'''
net[0]：选择 Sequential 中索引为0的模块（第一个 nn.Linear(4,8)）
.named_parameters()：返回该层参数的可迭代对象，每个元素是(参数名,参数张量)元组
列表推导式：提取每个参数的名称和形状
* 解包：将列表元素解开，作为多个独立参数传给 print()，实现单行输出
'''
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# net.named_parameters()：获取所有可训练参数（包括所有子模块）
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [6]:
'''
net.state_dict()：获取整个网络的参数字典（OrderedDict），键是参数名（如 '0.weight'、'2.bias'）
['2.bias']：通过键名访问第二层（索引2）的偏置参数
.data：提取参数的原始数据张量，不包含梯度追踪信息
'''
net.state_dict()['2.bias'].data

tensor([-0.1681])

从嵌套块收集参数

In [7]:
'''
X(2,4) 
  → block0: Linear(4→8) → ReLU → Linear(8→4) → ReLU
  → block1: 同上
  → block2: 同上
  → block3: 同上
  → Linear(4→1)
输出: (2,1)
'''
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())
# block2内部结构=block1→block1→block1→block1
def block2():
    net = nn.Sequential()
    # 将block1重复4次串联，形成一个深层子网络
    for i in range(4):
        # 动态添加子模块
        net.add_module(f'block {i}', block1()) 
    return net
'''
结构：block2（4层嵌套）+输出层（4→1）
总层数：4个子模块×2个Linear层+1个输出层=9个Linear层
输入输出：X（如(2,4)）→ 标量（如(2,1)）
'''
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3113],
        [-0.3113]], grad_fn=<AddmmBackward0>)

设计了网络后，我们看看它是如何工作的

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [9]:
'''
rgnet (Sequential)
└── [0] block2 (Sequential)
    ├── [0] block 0 (block1 → Sequential)
    ├── [1] block 1 (block1 → Sequential)  ← 选中这一层
    │   ├── [0] Linear(4, 8)               ← 再选这一层
    │   └── [1] ReLU()
    └── [2] block 2 (block1)
    └── [3] block 3 (block1)
└── [1] Linear(4, 1)
rgnet[0]：选择 Sequential 中第0个模块→即block2()（4个block1的容器）
rgnet[0][1]：在 block2 中索引为1的模块→即第2个 block1()
rgnet[0][1][0]：在该 block1 中索引为0的模块→即第一个nn.Linear(4,8)
.bias：获取该线性层的偏置参数对象（Parameter）
.data：提取底层数据张量（无梯度追踪）
'''

rgnet[0][1][0].bias.data

tensor([ 0.4338, -0.2437, -0.4322,  0.2146,  0.3343,  0.3795, -0.3809, -0.0874])

内置初始化

In [10]:
def init_normal(m):
    if type(m) == nn.Linear: # 检查模块是否为Linear层
        # # 权重初始化：正态分布N(0, 0.01²)（原地）
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias) # # 偏置初始化：全零（原地）
# 递归应用：此函数会被 apply() 自动传递给网络的每个子模块
net.apply(init_normal)
'''
net[0]：访问第一层（如 nn.Linear(4,8)）
.weight.data[0]：获取权重张量的第0行数据（形状 (8,)）
.bias.data[0]：获取偏置张量的第0个元素（标量）
返回值：(tensor([...]), tensor(0.))
'''
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0047, -0.0063, -0.0102, -0.0056]), tensor(0.))

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        # nn.init.constant_：原地将权重全部填充为指定值 1（所有元素相同）
        nn.init.constant_(m.weight, 1) # 所有权重设为常数1
        nn.init.zeros_(m.bias) # 偏置全零
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法

In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        # init_xavier：专为打破对称性设计，权重从[-a, a]区间均匀采样（a=sqrt(6/(fan_in + fan_out))）
        # 适合ReLU激活函数，防止梯度消失/爆炸
        nn.init.xavier_uniform_(m.weight) # Xavier均匀分布初始化
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 常数42初始化
# 应用于第0层（如 Linear(4,8)）,net[0] 所有 Linear 层 → Xavier初始化
net[0].apply(init_xavier)  
# 应用于第2层（如 Linear(8,1)）,net[2] 所有 Linear 层 → 常数42初始化
net[2].apply(init_42) 
print(net[0].weight.data[0]) # 查看第0层权重第0行
print(net[2].weight.data) # 查看第2层所有权重

tensor([ 0.5579, -0.2446,  0.1389,  0.6962])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        # 步骤1：均匀分布初始化
        nn.init.uniform_(m.weight, -10, 10) # 权重从U[-10,10]采样
        # 步骤2：稀疏化（保留大值，置零小值）
        '''
        创建布尔掩码（abs() >= 5），保留绝对值≥5的权重，其余置零
        效果：约50%的权重被置零（因为均匀分布中≥5的概率约10%，但绝对值≥5包含正负两个方向，所以总概率约20%。
        注意：如果区间是[-10,10]，则|w|≥5的概率是 (10-5)/10 = 0.5，即50%）
        实际上，对于 U[-10,10] 分布，|w| >= 5 的概率是 50%（因为区间一半是[-10,-5]∪[5,10]）
        最终结果：稀疏权重矩阵（约50%非零，50%为零）
        mask = (m.weight.data.abs() >= 5)  # 布尔掩码: True/False
        m.weight.data = m.weight.data * mask  # 乘法时True→1, False→0
        布尔张量在数值运算中会自动转为 0 和 1：
        True  转换为 1，weight * 1 = weight（保留原值）
        False  转换为 0，weight * 0 = 0（置零）
        '''
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2] # 查看第0层权重的前两行

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  7.7417, -9.3673, -8.4343],
        [ 5.8698,  0.0000,  0.0000, -9.7179]], grad_fn=<SliceBackward0>)

In [14]:
'''
net[0]：访问第0层（如 Linear(4, 8)）
.weight.data：获取权重参数的底层数据张量
[:]：全切片，选中所有元素
+= 1：原地操作，每个元素值 +1
'''
net[0].weight.data[:] += 1
# [0, 0]：访问第0行第0列的特定元素（第一个输出神经元对第一个输入特征的权重）
net[0].weight.data[0, 0] = 42
# [0]：获取第0行（即第一个输出神经元对应的所有输入权重）
# 返回值：形状为 (4,) 的张量
net[0].weight.data[0] # 显示第0行所有权值

tensor([42.0000,  8.7417, -8.3673, -7.4343])

参数绑定


In [15]:
shared = nn.Linear(8, 8)
# .Sequential：不会复制shared，而是保持引用（net[2]和net[4]指向同一块内存）
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), # shared：一个 Linear(8,8)实例对象
                    shared, nn.ReLU(), # 索引2：第1次使用
                    shared, nn.ReLU(), # 索引4：第2次使用（同一对象！）
                    nn.Linear(8, 1))
net(X)
# 比较两层的第0行权重
print(net[2].weight.data[0] == net[4].weight.data[0])
# 修改 net[2]的权重，net[4]同步变化
net[2].weight.data[0, 0] = 100
# 再次验证仍为 True，确认内存地址完全相同
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
